In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    # for filename in filenames:
        # print(os.path.join(dirname, filename))
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics

In [ ]:
!pip install ruamel.yaml

In [ ]:
import os
import random
import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
from IPython.display import Video
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
import pathlib
import glob
from tqdm.notebook import trange, tqdm
import warnings
import sys
import ruamel.yaml
warnings.filterwarnings('ignore')

In [ ]:
image = cv2.imread("/kaggle/input/smoke-fire-detection-yolo/data/train/images/AoF00000.jpg")
h, w, c = image.shape
print(f"The image has dimensions {w}x{h} and {c} channels.")

In [ ]:
model = YOLO("yolo12s.pt") 

In [ ]:
image = "/kaggle/input/smoke-fire-detection-yolo/data/train/images/AoF00012.jpg"
result_predict = model.predict(source = image, imgsz=(640))

In [ ]:
plot = result_predict[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

In [ ]:
path = "/kaggle/input/smoke-fire-detection-yolo"

In [ ]:
yaml = ruamel.yaml.YAML()
# yaml.preserve_quotes = True
with open("/kaggle/input/smoke-fire-detection-yolo/data.yaml") as fp:
    data = yaml.load(fp)

In [ ]:
data['path'] = path

In [ ]:
print(data)

In [ ]:
with open("data1.yaml", "w") as f:
    yaml.dump(data, f)

In [ ]:
Final_model = YOLO('yolo12s.yaml').load('yolo12s.pt')

In [ ]:
Result_Final_model = Final_model.train(
    data="/kaggle/working/data1.yaml",
    epochs=100,
    imgsz=640,
    batch=32,
    lr0=0.005,
    dropout=0.15,  # Для YOLO dropout задаётся в архитектуре модели, а не здесь
    # device='cpu',      # Должно быть строкой
    optimizer='AdamW',  # Указывается строкой
    # Добавьте параметры для ускорения:
    cache='ram',     # Кэширование данных
    workers=4,       # Оптимально для Kaggle
    close_mosaic=10, # Ускоряет обучение
    # pretrained=True  # Если используете предобученные веса
)

In [ ]:
list_of_metrics = ["P_curve.png","R_curve.png","confusion_matrix.png"]

for i in list_of_metrics:
    img = cv2.imread(f"/kaggle/working/runs/detect/train/{i}")
    plt.figure(figsize = (16, 12))
    plt.imshow(np.array(img))
    plt.show()

In [ ]:
Valid_model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

In [ ]:
Valid_model = YOLO('/kaggle/input/fire-smoke-weights/fire_smoke_params_yolo12s.pt')

In [ ]:
metrics = Valid_model.val(split = 'val')

In [ ]:
# final results 
print("precision(B): ", metrics.results_dict["metrics/precision(B)"])
print("metrics/recall(B): ", metrics.results_dict["metrics/recall(B)"])
print("metrics/mAP50(B): ", metrics.results_dict["metrics/mAP50(B)"])
print("metrics/mAP50-95(B): ", metrics.results_dict["metrics/mAP50-95(B)"])

In [ ]:
images = os.listdir("/kaggle/input/smoke-fire-detection-yolo/data/test/images")
for i in range(5, 15):
    image = os.path.join("/kaggle/input/smoke-fire-detection-yolo/data/test/images", images[i])
    result_predict = Valid_model.predict(source = image, imgsz=(640), iou=0.4)

    plot = result_predict[0].plot()
    plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(plot))